<a href="https://colab.research.google.com/github/kassymkhan0101/project_1/blob/master/Project_1_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Project 1 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [0]:

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [5]:
def get_list_of_university_towns():
  
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    utowns = []

    with open('university_towns.txt') as txt:
        for line in txt:
            if 'edit' in line:
                town = line.split('[')[0].strip()
            else:
                utowns.append((town, line.split('(')[0].strip()))
    df_utowns = pd.DataFrame.from_records(utowns)
    df_utowns.columns = ['State', 'RegionName']
    return df_utowns
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [6]:
def get_recession_start():
    gd = pd.ExcelFile('gdplev.xls')
    gd = gd.parse("Sheet1", skiprows=219)
    gd = gd[['1999q4', 9926.1]]
    gd.columns = ['quarter','gdp']
    k = len(gd)
    for i in range(0, k):
        if (gd.iloc[i][1] > gd.iloc[i+1][1]) and (gd.iloc[i+1][1] > gd.iloc[i+2][1]):
            return gd.iloc[i][0]
get_recession_start()  


'2008q3'

In [7]:
def get_recession_end():
    
    '''Returns the year and quarter of the recession end time as a
    string value in a format such as 2005q3'''
    gd = pd.ExcelFile('gdplev.xls')
    gd = gd.parse("Sheet1", skiprows=219)
    gd = gd[['1999q4', 9926.1]]
    gd.columns = ['quarter','gdp']
    first = get_recession_start()
    first_index = gd[gd['quarter'] == first].index.tolist()[0]
    gd=gd.iloc[first_index:]
    k = len(gd)
    for i in range(0, k):
        if (gd.iloc[i][1] < gd.iloc[i+1][1]) and (gd.iloc[i+1][1] < gd.iloc[i+2][1]):
            return gd.iloc[i+2][0]
get_recession_end()          

'2009q4'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gd = pd.ExcelFile('gdplev.xls')
    gd = gd.parse("Sheet1", skiprows=219)
    gd = gd[['1999q4', 9926.1]]
    gd.columns = ['quarter','gdp']
    x = gd.index[gd['quarter'] == get_recession_start()][0]
    y = gd.index[gd['quarter'] == get_recession_end()][0]
    return gd.iloc[gd.iloc[x:y+1, 1 ].idxmin(), 0]

get_recession_bottom()



'2009q2'

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    conv = pd.read_csv('City_Zhvi_AllHomes.csv')
    conv['State'] = conv['State'].map(states)
    conv.set_index(['State', 'RegionName'], inplace=True)
    conv = conv.loc[:,"2000-01":"2016-08"]
    new_columns = [str(u)+v for u in range(2000, 2017) for v in ['q1', 'q2', 'q3', 'q4']]
    new_columns = new_columns[:-1] 
    t = 0
    for c in new_columns:
        conv[c] = conv.iloc[:, t:t+3].mean(axis=1)
        t = t+3
    conv = conv.loc[:, '2000q1':]
    return conv
convert_housing_data_to_quarters().shape

(2500, 67)

In [11]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the null hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
   
    first = get_recession_start()
    last = get_recession_bottom()
    conv = convert_housing_data_to_quarters()
    conv = conv.loc[:, first: last]
    conv.reset_index(inplace=True)
    conv['price_ratio'] = (conv[first] - conv[last]) / conv[first]
    towns = get_list_of_university_towns()  
    towns_l = towns['RegionName'].tolist()
    conv['isUniTown'] = conv.RegionName.apply(lambda u: u in towns_l)
    uni_data =conv[conv.isUniTown].copy().dropna()
    not_uni_data = conv[~conv.isUniTown].copy().dropna()
    p_val = ttest_ind(uni_data['price_ratio'], not_uni_data['price_ratio'])[1]
    change = p_val < 0.01   
    verdict = 'university town' if uni_data['price_ratio'].mean() < not_uni_data['price_ratio'].mean() else 'non-university town'
    return (change, p_val, verdict)
run_ttest()

(True, 2.4264222248661247e-14, 'university town')